In [33]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from eugenenet.smallvggnetv2 import SmallVGGNet
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os



In [39]:
import pandas as pd

# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []
NUMBER_OF_SAMPLES = 50000

TRAIN_DATA_FILE = 'synimg/train/data.csv'
# grab the image paths and randomly shuffle them
df_imagePaths = pd.DataFrame.from_csv(TRAIN_DATA_FILE)
df_imagePaths_sample = df_imagePaths.sample(n=NUMBER_OF_SAMPLES)


[INFO] loading images...


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:11: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [40]:
# construct the image generator for data augmentation
#aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
#	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,#
#	horizontal_flip=True, fill_mode="nearest")
 
datagen=ImageDataGenerator(rescale=1./255., validation_split=0.15)
    


In [41]:
train_generator=datagen.flow_from_dataframe(
dataframe=df_imagePaths_sample,
directory="./",
x_col="filepath",
y_col="style_name",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,64))

valid_generator=datagen.flow_from_dataframe(
dataframe=df_imagePaths_sample,
directory=".",
x_col="filepath",
y_col="style_name",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,64))

Found 42500 validated image filenames belonging to 10 classes.
Found 7500 validated image filenames belonging to 10 classes.


In [42]:
# initialize our VGG-like Convolutional Neural Network
model = SmallVGGNet.build(width=64, height=32, depth=3, classes=len(train_generator.class_indices))

# initialize our initial learning rate, # of epochs to train for,
# and batch size
INIT_LR = 0.01
EPOCHS = 75
BS = 32
 
# initialize the model and optimizer (you'll want to use
# binary_crossentropy for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
 
# train the network
#H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS, epochs=EPOCHS)

#H = model.fit(trainX, trainY, steps_per_epoch=len(trainX) // BS, epochs=EPOCHS)

[INFO] training network...


In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

H = model.fit_generator(generator = train_generator, 
                        steps_per_epoch=STEP_SIZE_TRAIN, 
                        validation_data = valid_generator,  
                        validation_steps=STEP_SIZE_VALID, 
                        epochs=EPOCHS)

Epoch 1/75
1328/1328 [==============================] - 32s 24ms/step - loss: 1.7626 - acc: 0.3386 - val_loss: 1.3825 - val_acc: 0.4495
Epoch 2/75
1328/1328 [==============================] - 32s 24ms/step - loss: 1.1881 - acc: 0.5238 - val_loss: 0.9536 - val_acc: 0.6024
Epoch 3/75
1328/1328 [==============================] - 32s 24ms/step - loss: 0.9513 - acc: 0.6019 - val_loss: 0.8106 - val_acc: 0.6350
Epoch 4/75
1328/1328 [==============================] - 32s 24ms/step - loss: 0.8535 - acc: 0.6312 - val_loss: 0.8161 - val_acc: 0.6350
Epoch 5/75
1328/1328 [==============================] - 32s 24ms/step - loss: 0.8181 - acc: 0.6391 - val_loss: 0.7580 - val_acc: 0.6587
Epoch 6/75
1328/1328 [==============================] - 32s 24ms/step - loss: 0.8016 - acc: 0.6428 - val_loss: 0.7485 - val_acc: 0.6593
Epoch 7/75
1328/1328 [==============================] - 32s 24ms/step - loss: 0.7929 - acc: 0.6477 - val_loss: 0.7571 - val_acc: 0.6535
Epoch 8/75
1328/1328 [==========================

With batch normalisation: 
* 40 epochs: acc: 0.6613, loss: 0.7873
* 65 epochs: acc: ~0.70
* 75 epochs: acc: 0.7248


In [32]:
len(train_generator.class_indices)

10